# Unit decorators

### cost decorator

The cost decorator adds a free on board purchase cost based on the exponential scale up equation:

$$New\ cost = N \cdot cost \bigg(\frac{CE_{new}}{CE}\bigg) \bigg(\frac{S_{new}}{N \cdot S}\bigg)^{n}$$
    
$$Electricity\ rate = kW \bigg(\frac{S_{new}}{S}\bigg)$$

$$N = ceil \bigg( \frac{S_{new}}{ub} \bigg)$$

$N\text{: Number of units}$

$ub\text{: Upper bound of size}$

$CE_{new}\text{: New Chemical Engineering Plant Cost Index}$

$CE\text{: Chemical Engineering Plant Cost Index}$

$S_{new}\text{: New size value}$

$S\text{: Original size value}$

$cost\text{: Free on board purchase cost at size S}$

$kW\text{: Electricity rate in kW at size S}$

$n\text{: Exponential factor}$


Create a Shredder Unit subclass for sugar cane with the following exponential scale up factors [[1]](#References):

* Chemical Engineering Plant Cost Index: 567.3
* Size: 500,000 kg/hr flow rate
* Purchase cost: 2,500,000
* Electricity rate: 3,000 kW
* Exponential factor: 0.6

Additionally, include a bare module factor of 1.39 as an approximation for this example [[2]](#References).

In [1]:
import biosteam as bst
from biosteam.units.decorators import cost

# Set up thermo property package
chemicals = bst.Chemicals(['Water', 'Ethanol'])
SugarCane = bst.Chemical.blank('SugarCane', phase_ref='s')
SugarCane.default()
chemicals.append(SugarCane)
bst.settings.set_thermo(chemicals)

# For year 2018
bst.CE = 603.1 

# basis will be the total flow rate in kg/hr
@cost('Flow rate', units='kg/hr', cost=2.5e6,
      CE=567.3, n=0.6, S=500e3, kW=3000, BM=1.39)
class Shredder(bst.Unit): pass
# Units without a `_run` method assume one input and output stream
# that share the same flow rates and conditions

### Test a Shreadder object ###

# Display flow rate in kg/hr
bst.Stream.display_units.flow = 'kg/hr'

feed = bst.Stream(SugarCane=1e6, units='kg/hr')
shredder = Shredder(ins=feed)
shredder.simulate()

shredder.show()
shredder.results()

Shredder: U1
ins...
[0] s1
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow (kg/hr): SugarCane  1e+06
outs...
[0] s2
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow (kg/hr): SugarCane  1e+06


Shredder                        Units       U1
Power               Rate           kW    6e+03
                    Cost       USD/hr      469
Design              Flow rate   kg/hr    1e+06
Purchase cost       Shredder      USD 4.03e+06
Total purchase cost               USD 4.03e+06
Utility cost                   USD/hr      469

Cost data is stored as CostItem objects in the `cost_items` dictionary:

In [2]:
Shredder.cost_items

{'Shredder': <CostItem: Flow rate (kg/hr)>}

In [3]:
Shredder.cost_items['Shredder']

CostItem: Flow rate (kg/hr)
 S     5e+05
 ub    0
 CE    567
 cost  2.5e+06
 n     0.6
 kW    3e+03
 BM    1.39



Any option can be changed:

In [4]:
Shredder.cost_items['Shredder'].cost = 3e6 # Change base cost
# This also works:
# Shredder.cost_items['Shredder']['cost'] = 3e6
shredder.simulate()
shredder.results()

Shredder                        Units       U1
Power               Rate           kW    6e+03
                    Cost       USD/hr      469
Design              Flow rate   kg/hr    1e+06
Purchase cost       Shredder      USD 4.83e+06
Total purchase cost               USD 4.83e+06
Utility cost                   USD/hr      469

In [5]:
Shredder.cost_items['Shredder'].ub = 6e5 # Change size upper bound
shredder.simulate()
shredder.results()

Shredder                        Units       U1
Power               Rate           kW    6e+03
                    Cost       USD/hr      469
Design              Flow rate   kg/hr    1e+06
                    #Shredder                2
Purchase cost       Shredder      USD 6.38e+06
Total purchase cost               USD 6.38e+06
Utility cost                   USD/hr      469

Note: It is also possible to decorate a Unit subclass multiple times to add multiple cost items.

### References

<a id='References'></a>

1. Huang, H., Long, S., & Singh, V. (2016) "Techno-economic analysis of biodiesel and ethanol co-production from lipid-producing sugarcane" Biofuels, Bioproducts and Biorefining, 10(3), 299–315. https://doi.org/10.1002/bbb.1640

2. Seider, W. D., Lewin,  D. R., Seader, J. D., Widagdo, S., Gani, R., & Ng, M. K. (2017). Product and Process Design Principles. Wiley. Cost Accounting and Capital Cost Estimation (Chapter 16)

